In [1]:
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import pandas as pd
import re

from lib import SQLiteConnection
from lib import get_info_video_judobase
from lib import load_chromedriver

### Battles Competidor Obtener

In [2]:
driver = load_chromedriver(en_casa=False)

In [3]:
conn = SQLiteConnection('../judo.sql')

In [4]:
competitors = conn.as_pandas('select * from competitors where category="men_66"', index_col='index', parse_dates=['last_extraction'])

In [9]:
s='''
1		SHMAILOV Baruch	 (ISR)	1. seed (4. place wrl)	profile
2		FLICKER Tal	 (ISR)	2. seed (5. place wrl)	profile
3		AKHADOV Shakhram	 (UZB)	3. seed (22. place wrl)	profile
4		SHERSHAN Dzmitry	 (BLR)	4. seed (26. place wrl)	profile
5		NURILLAEV Sardor	 (UZB)	5. seed (27. place wrl)	profile
6		GAITERO MARTIN Alberto	 (ESP)	6. seed (29. place wrl)	profile
7		VAN GANSBEKE Kenneth	 (BEL)	7. seed (31. place wrl)	profile
8		CRISOSTOMO Joao	 (POR)	8. seed (33. place wrl)	profile
9		SOARES Andre	 (POR)		profile
10		SANTOS Diego	 (BRA)		profile
11		KATZ Nathan	 (AUS)		profile
12		SAYWELL Jeremy	 (MLT)		profile
13		BUNESCU Vadim	 (MDA)		profile
14		IADOV Bogdan	 (UKR)		profile
15		SAFAROV Orkhan	 (AZE)		profile
16		POLIAK Matej	 (SVK)		profile
17		MOGOPA Gavin	 (BOT)		profile
18		BOUCHER Mathias	 (FRA)		profile
19		CHANDLER Cole	 (USA)		profile
20		TEMELKOV Bozhidar	 (BUL)		profile
21		CHASYGOV Ismail	 (RUS)		profile
22		SPIKERMANN Tomas	 (ARG)		profile
23		KHAMETOV Islam	 (RUS)		profile
24		LOMBARDO Manuel	 (ITA)		profile
25		MEDVES Matteo	 (ITA)		profile
26		ALIYEV Ibrahim	 (AZE)		profile
27		VAN HARTEN Matthijs	 (NED)		profile
28		KOSSEKOV Muhammet	 (TKM)		profile
29		MENAGED Yarin	 (ISR)		profile
30		FOCA Nicolae	 (MDA)		profile
31		GARCIA TORNE David	 (ESP)		profile
32		DANTE Amir	 (ISR)		profile
33		VALOIS Jacob	 (CAN)		profile
34		TURSUNALIEV Ruslan	 (KGZ)		profile
'''

In [5]:
s = '''1		COHEN Gili	 (ISR)	1. seed (11. place wrl)	profile
2		DELGADO Angelica	 (USA)	2. seed (12. place wrl)	profile
3		KELMENDI Majlinda	 (KOS)	3. seed (15. place wrl)	profile
4		PRIMO Gefen	 (ISR)	4. seed (18. place wrl)	profile
5		GILES Chelsie	 (GBR)	5. seed (19. place wrl)	profile
6		BABAMURATOVA Gulbadam	 (TKM)	6. seed (29. place wrl)	profile
7		EASTON Tinka	 (AUS)	7. seed (30. place wrl)	profile
8		LOPEZ SHERIFF Estrella	 (ESP)	8. seed (33. place wrl)	profile
9		CHITU Andreea	 (ROU)		profile
10		KOCHER Fabienne	 (SUI)		profile
11		KUZNETSOVA Alesya	 (RUS)		profile
12		ZIYAEVA Gulnoza	 (UZB)		profile
13		BOBRIKOVA Daria	 (RUS)		profile
14		JANASHVILI Mariam	 (GEO)		profile
15		CESAR Taciana	 (GBS)		profile
16		FOFANA Salimata	 (CIV)		profile
17		KAKKO Katri	 (FIN)		profile
18		KELDIYOROVA Diyora	 (UZB)		profile
19		HOGREFE ACEA Sofia	 (ESP)		profile
20		CASTAGNOLA Martina	 (ITA)		profile
21		MAMMADALIYEVA Gultaj	 (AZE)		profile
22		PIMENTA Larissa	 (BRA)		profile
23		KAFRI Paz	 (ISR)		profile
24		MOKDAR Faiza	 (FRA)		profile
25		PUREVSUREN Buyankhishig	 (AZE)		profile
26		LEVYTSKA-SHUKVANI Tetiana
'''

In [10]:
comps = []

In [11]:
for line in s.split('\n'):
    try:
        comps.append(line.split('\t')[2])
    except:
        print(line)

In [12]:
comps[:5]

['SHMAILOV Baruch',
 'FLICKER Tal',
 'AKHADOV Shakhram',
 'SHERSHAN Dzmitry',
 'NURILLAEV Sardor']

In [13]:
i=0

In [11]:
i = comps.index('KOCHER Fabienne')
i

9

In [15]:
for nombre in comps[i:]:
    print(nombre)

    battles_comp = conn.as_pandas(f'select * from battles where local=="{nombre}"', parse_dates=['date'], 
                              index_col='index').reset_index(drop=True).sort_values('date', ascending=False)

    all_actions = []

    for battle in battles_comp.itertuples():
        if battle.url_video is None:
            continue
        else:
            actions_battle = get_info_video_judobase(battle.local, driver, battle.url_video)
            actions_battle = (a + (battle.event,) for a in actions_battle)
            all_actions.extend(actions_battle)

    df = pd.DataFrame(all_actions, columns=('opponent', 'you', 'action', 'action_detail', 'time', 'url_youtube', 'event'))

    df['local'] = nombre
    cols_order = [df.columns[-1]] + list(df.columns[:-1])
    df = df[cols_order]


    conn.add_table('actions', df, if_exists='append', index=False)

#    print('error con ', nombre)

SHMAILOV Baruch
FLICKER Tal
AKHADOV Shakhram
SHERSHAN Dzmitry
NURILLAEV Sardor
GAITERO MARTIN Alberto


KeyboardInterrupt: 

In [44]:
conn.as_pandas('select * from actions').local.unique()

array(['COHEN Gili', 'LOPEZ SHERIFF Estrella', 'GAITERO MARTIN Alberto'],
      dtype=object)

# Más

In [10]:
videos_estre = conn.as_pandas('select * from videos_info where local like "%SHERIFF%"')

In [11]:
videos_estre

,url_video,url_youtube,local,opponent
0,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=8b7mZQsTmK0,LOPEZ SHERIFF Estrella,PARK Da Sol
1,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=HdqymnSiloc,LOPEZ SHERIFF Estrella,JIMENEZ Kristine
2,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=JgPOEHOJ2jo,LOPEZ SHERIFF Estrella,GNETO Astride
3,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=xpd4OZFzESk,LOPEZ SHERIFF Estrella,TSCHOPP Evelyne
4,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=FJmiFvoi8Ro,LOPEZ SHERIFF Estrella,JIMENEZ Kristine
5,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=RfaQdGW0NwY,LOPEZ SHERIFF Estrella,KOSE Mai
6,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=U00rk9umAkY,LOPEZ SHERIFF Estrella,KRASNIQI Distria
7,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=kLIefZ6V2L4,LOPEZ SHERIFF Estrella,KUTSENKO Aigul
8,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=WSa7eH5Vv9U,LOPEZ SHERIFF Estrella,THUMM Verena
9,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=hJaATsCUcwM,LOPEZ SHERIFF Estrella,BEZALEL Ariel


### Filter and extract some videos

problema: cuando falla uno salta una pestaña y jode el resto...

In [8]:
some_videos = all_battles[all_battles.local=='WADA Rinoko'].url_video

for i, url_video in enumerate(some_videos):
    if i % 20 == -1:
        print(i)
    
    if url_video is None:
        continue
    # else
    
    try:
        local, opponent, url_youtube = get_info_video_judobase(driver, url_video)
    except:
        local, opponent, url_youtube = ('error',) * 3
        
    # assert 'https://www.youtube.com/watch?' in url_youtube, f'Strange url_youtube: {url_youtube}'

    conn.query(
        f'''
        INSERT OR IGNORE INTO videos_info 
        (url_video, url_youtube, local, opponent) 
        VALUES
        ("{url_video+'bb'}", "{url_youtube}", "{local}", "{opponent}");
        '''
    )

In [11]:
videos_estre

,url_video,url_youtube,local,opponent
0,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=8b7mZQsTmK0,LOPEZ SHERIFF Estrella,PARK Da Sol
1,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=HdqymnSiloc,LOPEZ SHERIFF Estrella,JIMENEZ Kristine
2,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=JgPOEHOJ2jo,LOPEZ SHERIFF Estrella,GNETO Astride
3,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=xpd4OZFzESk,LOPEZ SHERIFF Estrella,TSCHOPP Evelyne
4,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=FJmiFvoi8Ro,LOPEZ SHERIFF Estrella,JIMENEZ Kristine
5,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=RfaQdGW0NwY,LOPEZ SHERIFF Estrella,KOSE Mai
6,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=U00rk9umAkY,LOPEZ SHERIFF Estrella,KRASNIQI Distria
7,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=kLIefZ6V2L4,LOPEZ SHERIFF Estrella,KUTSENKO Aigul
8,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=WSa7eH5Vv9U,LOPEZ SHERIFF Estrella,THUMM Verena
9,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=hJaATsCUcwM,LOPEZ SHERIFF Estrella,BEZALEL Ariel


### Do all video_urls remaining

In [57]:
proposed_videos = all_battles.loc[all_battles.category == 'men_73', 'url_video']

In [58]:
proposed_videos

Series([], Name: url_video, dtype: object)

In [20]:
all_videos = conn.as_pandas('select * from videos_info')

In [23]:
for v in all_videos.url_video:
    print(v)

NameError: name 'all_videos' is not defined

In [32]:
all_battles.url_video.values.isin(all_videos.url_video.values).sum()

AttributeError: 'numpy.ndarray' object has no attribute 'isin'

In [29]:
all_videos.url_video.isin(all_battles.url_video)

0    False
1    False
Name: url_video, dtype: bool

In [24]:
all_battles.url_video.isin(all_videos.url_video).sum()

0

In [35]:
all_videos

NameError: name 'all_videos' is not defined

### Extraer info y acceder al youtube

### Extraer info

hay que juntarlo con extraccion url

In [38]:
videos_estre.url_video.str.slice(-2,)

0     bb
1     bb
2     bb
3     bb
4     bb
5     bb
6     bb
7     bb
8     bb
9     bb
10    bb
11    bb
12    bb
Name: url_video, dtype: object

In [32]:
urll = videos_estre.url_video.iloc[1]

In [39]:
urll = urll[:-2]

In [41]:
driver.get(urll)

In [42]:
# tabla con puntuaciones etiquetadas en tiempo
tabla_timed_points = driver.find_elements_by_tag_name('tbody')[1]

In [43]:
filas = tabla_timed_points.find_elements_by_tag_name('tr')

In [44]:
nrows = len(filas[0].find_elements_by_tag_name('td'))
nrows

3

In [45]:
if nrows == 2:
    for row_number, row in enumerate(filas):
        for col_number, value in enumerate(row.find_elements_by_tag_name('td')):
            print(row_number, col_number, value.text)

else:
    for row_number, row in enumerate(filas):
        for col_number, value in enumerate(row.find_elements_by_tag_name('td')):
            print(row_number, col_number, value.text)

    print(f'{nrows} rows!')

0 0 Waza-ari
0 1 00:09
0 2 
1 0 
1 1 02:36
1 2 Shido
Shido / Non-Combativity
3 rows!


In [46]:
seconds = 22

In [47]:
url_youtube_time = url_youtube + '&t=' + str(seconds)
url_youtube_time

'https://www.youtube.com/watch?v=3ajXhPeC9dQ&t=22'

In [48]:
driver.get(url_youtube_time)

### Youtube downloads

In [49]:
url_youtube = videos_estre.iloc[0,1]

In [50]:
url_youtube

'https://www.youtube.com/watch?v=8b7mZQsTmK0'

In [51]:
from pytube import YouTube

In [52]:
y = YouTube(url_youtube)

In [53]:
video = y.streams.filter(res='720p', mime_type='video/mp4').first()

In [54]:
assert video is not None, 'No video'

In [55]:
%time video.download(filename='estre720')

CPU times: user 2.97 s, sys: 2.28 s, total: 5.25 s
Wall time: 9.88 s


'/home/sheriff/Documents/projects/judo/judo/estre720.mp4'

### Edición con moviepy

In [35]:
3

3

In [56]:
import requests

In [3]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, concatenate_videoclips

In [2]:
text0 = TextClip("Primer chunk", font='Bookman-Demi', fontsize=70, color='red').set_position(('right', 'top'), relative=True).set_duration(5)


[MoviePy] This command returned an error !

OSError: MoviePy Error: creation of None failed because of the following error:

convert: delegate library support not built-in '/usr/share/fonts/type1/gsfonts/b018015l.pfb' (Freetype) @ warning/annotate.c/RenderFreetype/1797.
convert: no decode delegate for this image format `' @ error/constitute.c/ReadImage/556.
convert: no images defined `PNG32:/tmp/tmp6dqv06yt.png' @ error/convert.c/ConvertImageCommand/3300.
.

.This error can be due to the fact that ImageMagick is not installed on your computer, or (for Windows users) that you didn't specify the path to the ImageMagick binary in file conf.py, or that the path you specified is incorrect

In [3]:
text1 = TextClip("Segundo chunk", fontsize=70, color='blue').set_position((0.4, 0), relative=True).set_duration(5)
# text2 = TextClip("Tercer chunk", fontsize=70, color='green').set_position((0.8, 0), relative=True).set_duration(5)

# textos = (text0, text1, text2)

SyntaxError: invalid syntax (<ipython-input-3-f2f427ef38e1>, line 1)

In [2]:
clip = VideoFileClip('./estre720.mp4')

In [3]:
clip.duration

353.85

In [4]:
corte = clip.subclip(40,50)

In [64]:
import pkg_resources

In [65]:
pkg_resources.get_distribution('moviepy')

moviepy 0.2.3.5 (/home/sheriff/anaconda3/envs/video/lib/python3.7/site-packages)

In [63]:
3+3

6

In [13]:
textos = concatenate_videoclips([text0, text1])

In [15]:
final_clip = CompositeVideoClip([corte, textos])

In [16]:
final_clip.duration

4

In [17]:
final_clip.write_videofile("my_concatenatio2n2.mp4")

[MoviePy] >>>> Building video my_concatenatio2n2.mp4
[MoviePy] Writing audio in my_concatenatio2n2TEMP_MPY_wvf_snd.mp3


100%|█████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 271.76it/s]


[MoviePy] Done.
[MoviePy] Writing video my_concatenatio2n2.mp4


 99%|███████████████████████████████████████████████████████████████████████████████▏| 100/101 [00:06<00:00, 12.44it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: my_concatenatio2n2.mp4 

